<h3>The example code below illustrates how to handle events with N5990A Valiframe automation.
<br>These examples suppress all windows dialog prompts and instead redirect messages to the Python prompt</h3>
*For a more general overview of automation of Agilent Instruments using Python including N5990A ValiFrame [see here](Python_Automation.ipynb)*
<br>
<br>
First we will make our imports and connections to the ValiFrame environment.

In [1]:
import time
import win32api
import sys
sys.path.append('C:\\Program Files (x86)\\BitifEye\\ValiFrame\\PCI-Express3') # Add the location of the Valiframe dll's to the system path
import clr # Import the Common Runtime Library Python module

clr.AddReference("ValiFrameRemote") # Create a reference from CLR to the ValiFrameRemote DLL
clr.AddReference("VFBase") # Create a reference from CLR to the VFBase DLL
clr.AddReference("VFSequence") # Create a reference from CLR to the VFSequence DLL
clr.AddReference("VFUserInterface") # Create a reference from CLR to the VFUserInterface DLL

from BitifEye.ValiFrame.ValiFrameRemote import * # Import the entire ValiFrameRemote namespace from the DLL
from BitifEye.ValiFrame.Base import * # Import the entire Base namespace from the DLL
from BitifEye.ValiFrame.Sequence import * # Import the entire Sequence namespace from the DLL
from BitifEye.ValiFrame.UserInterface import * # Import the entire UserInterface namespace from the DLL
from BitifEye.Controls import * # Import the entire Controls namespace from the DLL

Next we need to create functions to handle what happens when Valiframe creates an event.
<br>In the example below we choose to handle events in the following way:
<ul>
    <li>**Log Events:** There are many events logged during procedure runs and it is undesirable to print these out or pause the program flow. We will simply ignore them using the **pass** statement.
    <li>**All Other Events:** We use the **raw_input()** function to pause the program allowing messages to be printed to the interpreter. The user must hit enter to continue the program flow
</ul>

In [2]:
def my_LogChanged(logentry):
    pass

def my_StatusChanged(source,status):
    print('Status Changed: ' + status)

def my_ProcedureCompleted(procedure,xmlresult):
    raw_input('Procedure Completed: ' + procedure + ' ' + xmlresult)

def my_DialogPopUp(source, args):
    #args.Dialog.ShowDialog()  #Uncomment to show Valiframe dialog box for all cases
    msgbox = args.DialogText

    # Definition for args.DialogType
    #Member name           Value   Description 
    #Form                  0       General Form if the dialog is not one of the other dialog types  
    #MessageBox            1       Standard Windows.Forms.MessageBox.  
    #ConnectionDialog      2       Connection dialog. The dialog which pops up if a connection change is required.  
    #UserInformationDialog 3       User Information Dialog, which contains a text and one button.  
    #UserDecisionDialog    4       User Decision Dialog, which contains one text and 2 buttons.  
    #InfoDialog            5       Info Dialog, which is the same as the UserInformationDialog (obsolete, will be removed in one of the next releases).  

    if args.DialogType == 0:
            raw_input('General Form Dialog: '+str(msgbox))
            #  add actions here.....
            #  args.Dialog.ShowDialog() #Uncomment to show dialog box for this case
    elif args.DialogType == 1:
            raw_input('Standard Windows.Forms.MessageBox: '+str(msgbox))
            #  add actions here.....
            #  args.Dialog.ShowDialog() #Uncomment to show dialog box for this case
    elif args.DialogType == 2:
            raw_input('Connection Dialog: '+str(msgbox))
            #  add actions here.....
            #  args.Dialog.ShowDialog() #Uncomment to show dialog box for this case
    elif args.DialogType == 3:
            raw_input('UserInformationDialog: '+str(msgbox))
            #  add actions here.....
            #  args.Dialog.ShowDialog() #Uncomment to show dialog box for this case
    elif args.DialogType == 4:
            raw_input('UserDecisionDialog: '+str(msgbox))
            #  add actions here.....
            #  args.Dialog.ShowDialog() #Uncomment to show dialog box for this case
    elif args.DialogType == 5:
            raw_input('InfoDialog: '+str(msgbox))
            #  add actions here.....
            #  args.Dialog.ShowDialog() #Uncomment to show dialog box for this case
    else:
            raw_input('Message not handled: '+str(msgbox))

Now we need to create an instance of the ValiFrameRemote object

In [17]:
my_vf_pcie = ValiFrameRemote() # Creates an instance of the ValiFrameRemote class

Next we need to register the event ValiFrame event handlers with the functions created above

In [18]:
#register event handlers
my_vf_pcie.LogChanged += LogChangedEventHandler(my_LogChanged);
my_vf_pcie.StatusChanged += StatusChangedEventHandler(my_StatusChanged);
my_vf_pcie.ProcedureCompleted += ProcedureCompletedEventHandler(my_ProcedureCompleted);
my_vf_pcie.DialogPopUp += DialogShowEventHandler(my_DialogPopUp);

Now we launch the application & load a config

In [19]:
my_vf_pcie.InitApplication("PciExpress3") # Initialize the application
my_vf_pcie.LoadProject("my_PCIe_proj.vfp")

We can query the available tests & IDs.

In [20]:
procedureIDs = procedureNames = [] # The variable names must be assigned to something but are overwritten in the next call
_, procedureIDs, procedureNames = my_vf_pcie.GetProcedures(procedureIDs, procedureNames)
procedureIDs = [int(id) for id in procedureIDs] # Convert to a python list of integers
procedureNames = [str(name) for name in procedureNames] # Convert to a python list of strings
procs = zip(procedureIDs, procedureNames) # Zip the id's & names together. A dictionary may also be a useful way to store these.
for proc_id,proc_name in procs:
    print proc_id, proc_name

309629 Pre-Shoot Calibration
309630 De-Emphasis Calibration
309620 Equalization Preset Calibration
309621 Equalization Custom Preset Calibration
309622 Random Jitter Calibration
309623 Sinusoidal Jitter Calibration
309625 DM Sinusoidal Interference Calibration
309626 Eye Height and Width Calibration
309627 Compliance Eye Calibration
328820 Preset Compliance Test 2.8
328821 Compliance Test 2.8
328831 Compliance Test 2.10
328811 Compliance Test 2.10
367260 Unit Interval 
367261 Template Tests 
367262 Eye-Width 
367263 Peak Differential Output Voltage 


In [7]:
procs = [309629,309630]

In [9]:
import System

In [11]:
procs_list = System.Array.CreateInstance(System.Int32,2)

In [12]:
i=0
for proc in procs:
    procs_list.SetValue(System.Int32(proc), i)
    i+=1

In [13]:
for proc in procs_list:
    print proc

309629
309630


In [14]:
results = my_vf_pcie.RunProcedures(procs_list,"ProcNames")

Connection Dialog: {\colortbl ;\red0\green0\blue255;\red0\green128\blue0;\red255\green128\blue0;\red255\green0\blue0;\red40\green130\blue100;\red204\green170\blue0;\red125\green33\blue255;}\tx150\fs18-\tab Connect M8020A Channel1 DATA OUT and DATA OUT BAR via N9398C blocking capacitors to oscilloscope channel 1 and 3. Use the same phase matched SMA/SMP cable pair that will be connected to the Rx lanes in the Rx Tests\cf2  and SMP/SMA adapters. \par \cf4-\tab Terminate all unused outputs as indicated in the connection diagram. \par 
Status Changed: Start
Status Changed: Station Init
Status Changed: Product PCI Express Init
Status Changed: Procedure Pre-Shoot Calibration Init
Status Changed: Procedure Pre-Shoot Calibration Start Iteration 0
Status Changed: Procedure Pre-Shoot Calibration Step 0
Status Changed: Procedure Pre-Shoot Calibration Step 1
Status Changed: Procedure Pre-Shoot Calibration CleanUp
Status Changed: Procedure Pre-Shoot Calibration Complete
Status Changed: Procedure De

<h4>Example 1: Synchronous RunProcedure() or RunProcedure() methods</h4>
<ul><li>The RunProcedure() and RunProcedures() methods are **synchronous** in the sense that the Procedure must complete before control is returned to the Python prompt.</ul>

In [15]:
# run procedures synchronously
#myprocs = array(309629,309630)
results = my_vf_pcie.RunProcedures(procedureIDs,"ProcNames")

Connection Dialog: {\colortbl ;\red0\green0\blue255;\red0\green128\blue0;\red255\green128\blue0;\red255\green0\blue0;\red40\green130\blue100;\red204\green170\blue0;\red125\green33\blue255;}\tx150\fs18-\tab Connect M8020A Channel1 DATA OUT and DATA OUT BAR via N9398C blocking capacitors to each Rx Lane selected for calibration P and N of the Riser Card 3.0 (CBBR) using phase matched SMA/SMP cable pairs. \par \cf1-\tab Connect Tx Lane 0 P and N of the CLB to oscilloscope channel 1 and 3 \cf1 using the N4235-61602 phase matched SMA/SMP cable pair (otherwise using a phase matched low loss cable pair). \par \cf0-\tab Plug the Compliance Load Board 3.0 into the Compliance Base Board 3.0 and the Compliance Base Board 3.0 into the Riser Card 3.0. \par \cf4-\tab Terminate all unused outputs as indicated in the connection diagram. \par 
Connection Dialog: {\colortbl ;\red0\green0\blue255;\red0\green128\blue0;\red255\green128\blue0;\red255\green0\blue0;\red40\green130\blue100;\red204\green170\blu

In the above example the program pauses at each line above waiting for the user to hit enter per our use of raw_input().

<h4>Example 2: Asynchronous StartRun() method</h4>
<ul><li>Since StartRun() returns control to the Python interpreter immediately we can insert a while loop to check the status of the procedures. This will allow the flow to execute just as the synchronous method above did.</ul>

In [26]:
# run procedures asynchronously
my_procedures = [309629, 309630]  ## define variable for procedures to execute
my_vf_pcie.SelectProcedures(procs_list)
my_vf_pcie.StartRun()
while True:
    time.sleep(2)
    status = my_vf_pcie.GetActualStatus()
    if status == 'Complete':
        break

Status Changed: Start
Status Changed: Start
Status Changed: Procedure Pre-Shoot Calibration Init
Status Changed: Procedure Pre-Shoot Calibration Init
Status Changed: Procedure Pre-Shoot Calibration Start Iteration 0
Status Changed: Procedure Pre-Shoot Calibration Start Iteration 0
Status Changed: Procedure Pre-Shoot Calibration Step 0
Status Changed: Procedure Pre-Shoot Calibration Step 0
Status Changed: Procedure Pre-Shoot Calibration Step 1
Status Changed: Procedure Pre-Shoot Calibration Step 1
Status Changed: Procedure Pre-Shoot Calibration CleanUp
Status Changed: Procedure Pre-Shoot Calibration CleanUp
Status Changed: Procedure Pre-Shoot Calibration Complete
Status Changed: Procedure Pre-Shoot Calibration Complete
Status Changed: Product PCI Express Complete
Status Changed: Product PCI Express Complete
Status Changed: Station Complete
Status Changed: Station Complete
Status Changed: Complete
Status Changed: Complete


In [16]:
# Unregister the event handlers
my_vf_mipi.LogChanged -= LogChangedEventHandler(my_LogChanged);
my_vf_mipi.StatusChanged -= StatusChangedEventHandler(my_StatusChanged);
my_vf_mipi.ProcedureCompleted -= ProcedureCompletedEventHandler(my_ProcedureCompleted);
my_vf_mipi.DialogPopUp -= DialogShowEventHandler(my_DialogPopUp);
# close object
my_vf_mipi.Finalize()

NameError: name 'my_vf_mipi' is not defined

Copyright © 2011 Agilent Technologies Inc. All rights reserved.

You have a royalty-free right to use, modify, reproduce and distribute this content (and/or any modified version) in any way you find useful, provided that you agree that Agilent has no warranty, obligations or liability.